# Simulación con Método de Monte-Carlo
## MA2004B - Optimización Estocástica

Steffany Mishell Lara Muy (A00838589)
*Instituto Tecnológico y de Estudios Superiores de Monterrey (ITESM)*

**Resumen**
El presente cuaderno de Python tiene como objetivo aplicar los conocimientos adquiridos en el módulo de simulación sobre el método de Montecarlo para estimar valores en problemáticas reales.

### Ejemplo 1

Supongamos que tenemos un satélite, que para su funcionamiento depende  de que al menos 2 paneles solares de los 5 que tiene disponibles estén en  funcionamiento, y queremos calcular φ la vida útil esperada del satélite (el  tiempo promedio de funcionamiento hasta que falla, usualmente conocido  en la literatura como MTTF - Mean Time To Failure).
Supongamos que cada panel solar tiene una vida útil que es aleatoria, y  está uniformemente distribuída en el rango [1000 hs, 5000 hs] (valor  promedio: 3000 hs).
Para estimar por Monte Carlo el valor de φ, haremos n experimentos, cada  uno de los cuales consistirá en sortear el tiempo de falla de cada uno de los  paneles solares del satélite, y observar cual es el momento en el cuál han  fallado 4 de los mismos, esta es la variable aleatoria cuya esperanza es el  tiempo promedio de funcionamiento del satélite.
El valor promedio de las n observaciones nos proporciona una estimación  de φ.

Table 1: Una simulación detallada con n = 6 experimentos.
De esta simulación, tenemos un valor estimado para la vida útil esperada  del satélite de 3683. Un indicador del error que podemos estar cometiendo  es la varianza o equivalentemente la desviación estándar de Sn, que en este  caso es (haciendo los cálculos) 297. 

Experimento vs Paneles hasta que falle:

| Experimento | 1     | 2     | 3     | 4     | 5     | Satelite x(i) |
|-------------|-------|-------|-------|-------|-------|---------------|
| 1           | 3027  | 1738  | 2376  | 4685  | 4546  | 4546          |
| 2           | 4162  | 4029  | 4615  | 3455  | 3372  | 4162          |
| 3           | 3655  | 2896  | 1378  | 4010  | 4144  | 4010          |
| 4           | 2573  | 2649  | 2117  | 3956  | 1281  | 2649          |
| 5           | 2977  | 2724  | 1355  | 2268  | 3262  | 2977          |
| 6           | 3756  | 4190  | 1749  | 3398  | 2581  | 3756          |
| Prom.       | -     | -     | -     | -     | -     | Sn/n = 3683   |

In [17]:
from tkinter import *
from tkinter import messagebox
import random

class app(Tk):
    def __init__(self):
        Tk.__init__(self)
        self.config(bg="lightblue",cursor="heart")
        self.geometry("800x400")
        self.title("Simulación Montecarlo - A0083589 Steffany Lara")

        self.l1 = Label(self, text="SIMULACIÓN MONTECARLO", bg="lightblue", font=("Times New Roman", 20,"bold"), fg="black")
        self.l1.pack(pady=10)

        self.l2 = Label(self, text="Cant. Variables Aleatorias Independientes:", bg="lightblue",fg="black")
        self.l2.place(x=10+140*0 , y=60*1)

        self.entry_variables_aleatorias = Entry(self, width=3)
        self.entry_variables_aleatorias.place(x=140*2 , y=30*2)

        self.l3 = Label(self, text="Cant. Experimentos:", bg="lightblue",fg="black")
        self.l3.place(x=10+140*0 , y=30*3)
        self.entry_experimentos = Entry(self, width=3)
        self.entry_experimentos.place(x=140*2 , y=30*3)

        self.l4 = Label(self, text="Criterio Sorteo:", bg="lightblue",fg="black")
        self.l4.place(x=10+140*0 , y=30*4)
        self.entry_sorteo = Entry(self, width=3)
        self.entry_sorteo.place(x=140*2 , y=30*4)

        self.l4 = Label(self, text="Criterio Sorteo:", bg="lightblue",fg="black")
        self.l4.place(x=10+140*0 , y=30*4)
        self.entry_sorteo = Entry(self, width=3)
        self.entry_sorteo.place(x=140*2 , y=30*4)


app().mainloop()

In [ ]:
class Montecarlo:
    def __init__(self, num_variables, num_experimentos, criterio_sorteo):
        self.num_variables = num_variables
        self.num_experimentos = num_experimentos
        self.criterio_sorteo = criterio_sorteo

    def sortear(self):
        resultados = []
        for _ in range(self.num_experimentos):
            experimento = [random.random() for _ in range(self.num_variables)]
            resultados.append(experimento)
        elegido = random.choice(resultados)
        return elegido

Importante: es una distribución normal :)

entrada: x
    numero paneles
    cant experimentos
    criterio de sorteo
    parametrizacion
        limite inferior
        limite superior
        media

proceso:
    1. crear un vector x(i) satelite que va a guardar el promedio de cada experimento 
    2. crear la funcion experimento que saca un tiempo hasta falla de la cada panel (numero_paneles)
    3. guardar en vector satelite el promedio del tiemop de falla por cada panel
    4. realizar ese experimento cant_experimento veces
    5. sacar el promedio del vector satelite
    6. sacar desviacion estandar muestral del vector satelite con base a limites y media
    7. sacar standard error.


salida:
    medias
    desviacion estandar muestral
    standard error


In [58]:
import random
import math


class Montecarlo:
    def __init__(self, num_variables=5, n_experimentos=6, criterio_sorteo=4,limite_inferior=1000,limite_superior=5000,media=3000): #paneles, experimentos, fallas, li,ls,media
        self.num_variables = num_variables
        self.n_experimentos = n_experimentos
        self.criterio_sorteo = criterio_sorteo
        self.limite_inferior = limite_inferior
        self.limite_superior = limite_superior
        self.media = media

        self.crear_experimentos()
        self.sortear()
        self.metricas()

        # experimentos, satelite (promedio de falla), desviacion estandar muestral, error estandar :)
        

    def crear_experimentos(self):
        ''' 
            Crea los experimentos n veces, cada experimento tiene num_variables variables aleatorias.
            "Cuenta" el tiempo hasta que falle cada panel (num_variables) paneles
        '''

        self.experimentos=[]
        for i in range(self.n_experimentos):
            experimento_n =[]
            for j in range(self.num_variables):
                tiempo_falla_panel=random.randint(self.limite_inferior,self.limite_superior)
                experimento_n.append(tiempo_falla_panel)
            self.experimentos.append(experimento_n)

      
    def sortear(self): 
        '''
            Es la que toma el criteio_sorteo y elige a partir de cual experimento falla el sistema (4 por defecto)
            y eso lo guarda en la variable satelite
        '''
        satelite = []
        for experimento in self.experimentos:
            experimento.sort()
            satelite.append(experimento[self.criterio_sorteo-1])

        self.satelite= satelite

    def metricas(self):
        '''
            Calcula las metricas de los experimentos realizados
                promedio de tiempo de falla del sistema (satelite)
                desviacion estandar muestral
                standard error
        '''
        self.promedio_falla =sum(self.satelite)/len(self.satelite)
        self.desviacion_estandar_muestral = math.sqrt(
            sum((x - self.media)**2 for x in self.satelite) / (len(self.satelite)-1))
        
        self.standard_error = self.desviacion_estandar_muestral /math.sqrt(len(self.satelite))

    def retornar_resultados(self):
        lista = [self.satelite,round(self.promedio_falla,2), round(self.desviacion_estandar_muestral,2) ,round(self.standard_error,2)]
        return self.experimentos,lista

simulacion1 = Montecarlo() #los parametros finales de li,ls y media son opcionales segun yo para el area de parametrizacion pero no entendí bien la tarea jaja
#print(experimentos)
print(simulacion1.retornar_resultados())



([[1483, 1662, 2765, 2886, 4541], [1121, 3380, 3619, 3880, 4692], [2030, 2322, 2737, 3351, 3357], [1091, 1340, 3719, 3850, 4500], [1842, 2910, 3045, 3097, 4422], [1881, 1886, 2687, 3415, 3701]], [[2886, 3880, 3351, 3850, 3097, 3415], 3413.17, 602.45, 245.95])


In [68]:
experimentos = [
    [3000, 2500, 4000, 3500, 4500],
    [2000, 3000, 2500, 4000, 5000],
    [1500, 3500, 3000, 4500, 4000]
]


for experimento in experimentos:
    experimento.sort()

print(experimentos)
print(len(experimentos[1]))

[[2500, 3000, 3500, 4000, 4500], [2000, 2500, 3000, 4000, 5000], [1500, 3000, 3500, 4000, 4500]]
5


In [ ]:
experimentos.insert({i+1} for i in experimentos)

In [67]:
conti=1
for experimento in experimentos:
    experimento.insert(0,conti)
    conti+=1

print(experimentos)

[[1, 2500, 3000, 3500, 4000, 4500], [2, 2000, 2500, 3000, 4000, 5000], [3, 1500, 3000, 3500, 4000, 4500]]
